# This example demonstrate how to train region classification (standart image classification model).

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Nomeroff-Net path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [3]:
# auto download latest dataset
from NomeroffNet.tools import modelhub

# download latest dataset
#info = modelhub.download_dataset_for_model("numberplate_options")
#PATH_TO_DATASET = info["dataset_path"]

# or local path dataset
PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "./data/dataset/OptionsDetector/numberplate_options_example")

In [4]:
PATH_TO_DATASET

'/mnt/data/var/www/nomeroff-net/./data/dataset/OptionsDetector/numberplate_options_example'

In [5]:
DATASET_NAME = "orientations"
VERSION = "2021_07_08_pytorch_lightning"

RESULT_PATH = os.path.join(NOMEROFF_NET_DIR, "./data/models/", 'numberplate_{}_{}.ckpt'.format(DATASET_NAME, VERSION))

In [6]:
from NomeroffNet import OrientationDetector

# definde your parameters
class CustomOrientationDetector(OrientationDetector):
    def __init__(self):
        OrientationDetector.__init__(self)
        
        # outputs 2
        self.orientations = [
            0,
            180
        ]
        
        
        self.epochs           = 1
        self.batch_size       = 1
        
        # count gpu for train
        self.gpus = 0
        
        self.height         = 64
        self.width         = 295


In [7]:
# initialize region detector.
npOrientationClassificator = CustomOrientationDetector()
npOrientationClassificator.prepare(PATH_TO_DATASET, verbose=1)

In [8]:
# train
model = npOrientationClassificator.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name       | Type        | Params
-------------------------------------------
0 | inp_conv   | Conv2d      | 896   
1 | conv1      | Conv2d      | 18.5 K
2 | conv2      | Conv2d      | 73.9 K
3 | conv3      | Conv2d      | 147 K 
4 | pool       | MaxPool2d   | 0     
5 | dropout    | Dropout     | 0     
6 | fc1        | Linear      | 2.1 M 
7 | fc2        | Linear      | 131 K 
8 | batch_norm | BatchNorm1d | 1.0 K 
9 | fc3        | Linear      | 514   
-------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.885     Total estimated model params size (MB)



[INFO] best model path /mnt/data/var/www/nomeroff-net/train/lightning_logs/version_35/checkpoints/epoch=0-step=0.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 1.0, 'test_loss': 0.31326183676719666}
--------------------------------------------------------------------------------


In [9]:
npOrientationClassificator.prepare(PATH_TO_DATASET, verbose=1)

In [10]:
npOrientationClassificator.test()


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 1.0, 'test_loss': 0.31326183676719666}
--------------------------------------------------------------------------------


[{'test_loss': 0.31326183676719666, 'test_accuracy': 1.0}]

In [11]:
npOrientationClassificator.save(RESULT_PATH)

model save to /mnt/data/var/www/nomeroff-net/./data/models/numberplate_orientations_2021_07_08_pytorch_lightning.ckpt


# Classification Report

In [12]:
npOrientationClassificator.load(RESULT_PATH)

NPOrientationNet(
  (inp_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
)

In [23]:
from NomeroffNet.data_modules.data_loaders import OrientationImgGenerator
from sklearn.metrics import classification_report
import torch

def orientation_classification_report(npOrientationClassificator, 
                                      path_dir=os.path.join(PATH_TO_DATASET, "test"), 
                                      mode_torch="cpu"):
    imageGenerator = OrientationImgGenerator(
        path_dir,
        npOrientationClassificator.width,
        npOrientationClassificator.height,
        npOrientationClassificator.batch_size,
        [len(npOrientationClassificator.class_region), len(npOrientationClassificator.count_lines), 2])
    imageGenerator.build_data()
    gen = imageGenerator.path_generator()
    
    out_idx_all = []
    label_idx_all = []
    for i, (img_paths, inputs, labels) in enumerate(gen, 0):
        labels = torch.from_numpy(labels)
        # get the inputs; data is a list of [inputs, labels]
        inputs = torch.from_numpy(inputs)
        if mode_torch == "gpu":
            inputs = inputs.cuda()
            npOrientationClassificator.model.cuda()
        with torch.no_grad():
            outputs = npOrientationClassificator.model(inputs)
        if mode_torch == "gpu":
            labels = labels.cuda()
        
        out_idx = torch.max(outputs, 1)[1].cpu().numpy()        
        label_idx = torch.max(labels, 1)[1].cpu().numpy()
        
        out_idx_all.extend(out_idx)
        label_idx_all.extend(label_idx)
    #print("out_idx_reg_all", out_idx_reg_all)
    print("\n\nclass region classification report:")
    print(classification_report(label_idx_all, 
                                out_idx_all, 
                                target_names=npOrientationClassificator.orientations))

In [24]:
orientation_classification_report(npOrientationClassificator)



class region classification report:


ValueError: Number of classes, 1, does not match size of target_names, 2. Try specifying the labels parameter